In [2]:
## phlquery.ipynb <-- jupyter version of the RAG-app; this is where it
##                    all started

## Load the API key
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
#from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [3]:
loaded_document = TextLoader("./data/PhilHistory.txt", encoding='utf-8').load()

In [4]:
print(loaded_document)

[Document(metadata={'source': './data/PhilHistory.txt'}, page_content='The Philippines, an archipelago comprising over 7,600 islands in Southeast Asia, boasts a profound and intricate history, shaped by millennia of indigenous development, centuries of colonial rule, and the unwavering spirit of its people. Its narrative is a mosaic of diverse cultures, struggles for sovereignty, and a continuous quest for national identity.\n\nI. The Pre-Colonial Tapestry (c. 900 AD - 1565 AD)\n\nLong before the arrival of European explorers, the Philippine islands were a vibrant hub of diverse societies. Early Filipinos, of Austronesian descent, arrived in waves, bringing with them advanced maritime technology, agricultural practices, and sophisticated social structures. Communities were typically organized into independent "barangays," named after the boats (balangay) that brought the early settlers. Each barangay was headed by a "datu" or "rajah," hereditary leaders who served as chief, judge, and 

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)
vector_db = FAISS.from_documents(chunks_of_text, OpenAIEmbeddings())

Created a chunk of size 1007, which is longer than the specified 1000
Created a chunk of size 1010, which is longer than the specified 1000


In [7]:
chunks_of_text

[Document(metadata={'source': './data/PhilHistory.txt'}, page_content='The Philippines, an archipelago comprising over 7,600 islands in Southeast Asia, boasts a profound and intricate history, shaped by millennia of indigenous development, centuries of colonial rule, and the unwavering spirit of its people. Its narrative is a mosaic of diverse cultures, struggles for sovereignty, and a continuous quest for national identity.\n\nI. The Pre-Colonial Tapestry (c. 900 AD - 1565 AD)'),
 Document(metadata={'source': './data/PhilHistory.txt'}, page_content='Long before the arrival of European explorers, the Philippine islands were a vibrant hub of diverse societies. Early Filipinos, of Austronesian descent, arrived in waves, bringing with them advanced maritime technology, agricultural practices, and sophisticated social structures. Communities were typically organized into independent "barangays," named after the boats (balangay) that brought the early settlers. Each barangay was headed by a

In [6]:
len(chunks_of_text)

19

In [ ]:

embeddings = OpenAIEmbeddings()
vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [8]:
vector_db

In [9]:
retriever = vector_db.as_retriever(search_kwargs={"k":3})

In [12]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D3F8737590>, search_kwargs={'k': 3})

In [10]:
response = retriever.invoke("How many islands does the Philippines have?")

In [11]:
response

[Document(metadata={'source': './data/PhilHistory.txt'}, page_content='The Philippines, an archipelago comprising over 7,600 islands in Southeast Asia, boasts a profound and intricate history, shaped by millennia of indigenous development, centuries of colonial rule, and the unwavering spirit of its people. Its narrative is a mosaic of diverse cultures, struggles for sovereignty, and a continuous quest for national identity.\n\nI. The Pre-Colonial Tapestry (c. 900 AD - 1565 AD)'),
 Document(metadata={'source': './data/PhilHistory.txt'}, page_content='Long before the arrival of European explorers, the Philippine islands were a vibrant hub of diverse societies. Early Filipinos, of Austronesian descent, arrived in waves, bringing with them advanced maritime technology, agricultural practices, and sophisticated social structures. Communities were typically organized into independent "barangays," named after the boats (balangay) that brought the early settlers. Each barangay was headed by a

## get the final answer using a simple LCEL

In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [13]:
template = """ Answer the question based only on the ff: context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [14]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [15]:
response = chain.invoke("How many islands does the Philippines have?")

In [19]:
response

'The Philippines comprises over 7,600 islands.'

In [16]:
#response = chain.invoke("How many islands does the Philippines have?")
#response = chain.invoke("When was Jose Riza executed?")
response = chain.invoke("Explain the galleon trade")

In [17]:
response

'The Galleon Trade was a trading route that linked Manila in the Philippines with Acapulco in Mexico for over 250 years. It involved the exchange of Asian goods like silk, spices, and porcelain for Mexican silver. This trade route brought significant wealth to a select few individuals, but often did not benefit the broader Filipino population.'